In [1]:
import pandas as pd
import numpy as np
import nba_api.stats.endpoints as ep
import datetime
import re
from os import path
import time
import pickle

In [2]:
def lshelper(game_arr, gd):
    for x in range(len(game_arr)):
        gid = game_arr[x, 4]
        match = game_arr[x, 6]
        pm = game_arr[x, 27]
        if game_arr[x, 7] == 'W':
            winner = game_arr[x, 2]
        else:
            winner = game_arr[x, 6][-3:]
        gd[gid] = (match, pm, winner)
    return gd


class StatBucket():

    def __init__(self):
        self.clog = None
        self.log = None
        self.oslog = None
        self.outcomes = None
        self.data = None

    def update_log(self, season):
        result = ep.leaguegamefinder.LeagueGameFinder(season_nullable=season)
        all_games = result.get_data_frames()[0]
        rs = all_games[all_games.SEASON_ID == '2' + season[:4]]
        rs = rs[rs.GAME_ID.str[:3] == '002'] #may need to update
        os = all_games[all_games.SEASON_ID == '4' + season[:4]]
        os = os[os.GAME_ID.str[:3] == '004']
        self.log = rs
        self.oslog = os
#         self.clog = rs.append(os)
        self.clog = pd.concat([rs,os])
    
    
    def get_log_stats(self):
        game_arr = self.log.to_numpy()
        os_arr = self.oslog.to_numpy()
        game_dat = dict()
        game_dat = lshelper(game_arr, game_dat)
        res = lshelper(os_arr, game_dat)        
        self.outcomes = res
        

def Season_csv(statfunc, gidset, playerdata=False):
    teams = []
    players= []
    count=0
    for gid in gidset:
        count+=1
        game = statfunc(game_id=gid).get_data_frames()
        players.append(game[0])
        teams.append(game[1])
#         if not (count%500):
#             with open(f'TEMP/players.bin', 'wb') as f:
#                 pickle.dump(players, f)
#             with open(f'TEMP/teams.bin', 'wb') as f2:
#                 pickle.dump(teams, f2)
    pstats = pd.concat(players)
    tstats = pd.concat(teams)
    return tstats, pstats


def append_game(statfunc, current_stats, gid, playerdata=False):
    if playerdata:
        game = statfunc(game_id=gid).get_data_frames()[0]
        current_stats = pd.concat([current_stats,game])
        return current_stats
    else:
        game = statfunc(game_id=gid).get_data_frames()[1]
        current_stats = pd.concat([current_stats,game])
        return current_stats
def split_gid(gidset, nos):
    rem = len(gidset)%nos
    relist = [int(len(gidset)/nos)] * nos
    relist[-1]+=rem
    return relist

In [3]:
# PLAYER DATA OR TEAM DATA
PLAYER_DATA = False


SEASONS = ['2021-22']

# ['2010-11',
#            '2011-12',
#            '2012-13',
#            '2013-14',
#            '2014-15',
#            '2015-16',
#            '2016-17',
#            '2017-18',
#            '2018-19',
#            '2019-20',
#            '2020-21',
#           '2021-22',
#           '2022-23']

# FN = ['fourfactors']

FN = ['scoring',
    'advanced',
    'fourfactors',
     'misc']


#      'scoring'
#      'summary',
#      'traditional'

FD = {'advanced':ep.boxscoreadvancedv2.BoxScoreAdvancedV2,
     'fourfactors':ep.boxscorefourfactorsv2.BoxScoreFourFactorsV2,
     'misc':ep.boxscoremiscv2.BoxScoreMiscV2,
     'scoring':ep.boxscorescoringv2.BoxScoreScoringV2,
     'summary':ep.boxscoresummaryv2.BoxScoreSummaryV2,
     'traditional':ep.boxscoretraditionalv2.BoxScoreTraditionalV2
    }

In [4]:
### FOR DIFFERENT SEASON_CSV_FUNCTION:

for i in range(len(SEASONS)):
    for j in FN:
        if path.exists(f'DATA/raw/teams/{j}/{j}{SEASONS[i]}.csv'):
            print(f"SKIPPING teams/{j}/{j}{SEASONS[i]}, already exists")
            continue
        sb = StatBucket()
        # THIS MAY NEED TO BE SEASONS[i][:4] for some earlier seasons??
        sb.update_log(SEASONS[i])
        sb.get_log_stats()
        gidset = set(sb.outcomes.keys())
        if not len(gidset):
            print(f'error with {SEASONS[i]}, no game ids')
            print(sb.outcomes)
            break
        
        if not path.exists(f'DATA/raw/log/log_{SEASONS[i]}.csv'):
            sb.clog.sort_values('TEAM_ID', inplace=True, kind='mergesort')
            sb.clog.sort_values('GAME_ID', inplace=True, kind='mergesort')
            sb.clog.to_csv(f'DATA/raw/log/log_{SEASONS[i]}.csv')
        
        nst, nsp = Season_csv(FD[j], gidset)
        nst.sort_values('TEAM_ID', inplace=True, kind='mergesort')
        nst.sort_values('GAME_ID', inplace=True, kind='mergesort')
        nst.to_csv(f'DATA/raw/teams/{j}/{j}{SEASONS[i]}.csv')
        nsp.sort_values('TEAM_ID', inplace=True, kind='mergesort')
        nsp.sort_values('GAME_ID', inplace=True, kind='mergesort')
        nsp.to_csv(f'DATA/raw/players/{j}/{j}{SEASONS[i]}.csv')


SKIPPING teams/scoring/scoring2021-22, already exists


ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)